In [1]:
import numpy as np
import pandas as pd
from collections import defaultdict
import networkx as nx
import seaborn as sns
import matplotlib.pyplot as plt
import re

//anaconda/lib/python3.5/site-packages/pandas/computation/__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


In [ ]:
def stable(rankings, A, B):
    r"""
    rankings[(a, n)] = partner that a ranked n^th
    >>> from itertools import product
    >>> A = ['1','2','3','4','5','6']
    >>> B = ['a','b','c','d','e','f']
    >>> rank = dict()
    >>> rank['1'] = (1,4,2,6,5,3)
    >>> rank['2'] = (3,1,2,4,5,6)
    >>> rank['3'] = (1,2,4,3,5,6)
    >>> rank['4'] = (4,1,2,5,3,6)
    >>> rank['5'] = (1,2,3,6,4,5)
    >>> rank['6'] = (2,1,4,3,5,6)
    >>> rank['a'] = (1,2,3,4,5,6)
    >>> rank['b'] = (2,1,4,3,5,6)
    >>> rank['c'] = (5,1,6,3,2,4)
    >>> rank['d'] = (1,3,2,5,4,6)
    >>> rank['e'] = (4,1,3,6,2,5)
    >>> rank['f'] = (2,1,4,3,6,5)
    >>> Arankings = dict(((a, rank[a][b_]), B[b_]) for (a, b_) in product(A, range(0, 6)))
    >>> Brankings = dict(((b, rank[b][a_]), A[a_]) for (b, a_) in product(B, range(0, 6)))
    >>> rankings = Arankings
    >>> rankings.update(Brankings)
    >>> stable(rankings, A, B)
    [('1', 'a'), ('2', 'b'), ('3', 'd'), ('4', 'f'), ('5', 'c'), ('6', 'e')]
    """
    partners = dict((a, (rankings[(a, 1)], 1)) for a in A)
    is_stable = False # whether the current pairing (given by `partners`) is stable
    while is_stable == False:
        is_stable = True
        for b in B:
            is_paired = False # whether b has a pair which b ranks <= to n
            for n in range(1, len(B) + 1):
                a = rankings[(b, n)]
                a_partner, a_n = partners[a]
                if a_partner == b:
                    if is_paired:
                        is_stable = False
                        partners[a] = (rankings[(a, a_n + 1)], a_n + 1)
                    else:
                        is_paired = True
    return sorted((a, b) for (a, (b, n)) in partners.items())

In [2]:
def create_path(element):
    path=[]
    path.append('start')
    
    visits = element[1]
    visits = sorted(visits,key=lambda x:x[3])
    
    channels = [x[1] for x in visits]
    for channel in channels:
        path.append(channel)
    
    conversion = sum([x[2] for x in visits])>0
    if conversion:
        path.append('conversion')
    else:
        path.append('null')
        
    return path

In [3]:
def create_path_with_value(element):
    path=[]
    path.append('start')
    
    visits = element[1]
    visits = sorted(visits,key=lambda x:x[3])
    
    channels = [x[1] for x in visits]
    for channel in channels:
        path.append(channel)
    
    conversion = sum([x[2] for x in visits])>0
    if conversion:
        path.append('conversion')
    else:
        path.append('null')
    
    value = sum([x[4] for x in visits])
    path.append(str(value))
    
        
    return path 

In [4]:
def create_last_click_stats_pair(element):
    visits = element[1]
    visits = sorted(visits,key=lambda x:x[3],reverse=True)
    
    conversion = sum(x[2] for x in visits)>0
    if conversion:
        result = []
        channel = visits[0][1]
        value = sum(x[4] for x in visits)
        result.append((channel,value))
        return result
    else:
        return []

In [5]:
def create_first_order_states_pair(path):
    if len(path)<3:
        return []
    else:
        result = []
        
        for i in range(0,len(path)-1):
            from_state = path[i]
            to_state = path[i+1]
            result.append(((from_state,to_state),1))
        
        return result

In [6]:
def create_first_order_states_pair_with_value(path):
    if len(path)<3:
        return []
    else:
        result = []
        value = float(path[-1])
        for i in range(0,len(path)-2):
            from_state = path[i]
            to_state = path[i+1]
            result.append(((from_state,to_state),value))
        
        return result   

In [7]:
channels = ["banner","text","keyword","link","video","mobile",'unknown']

In [8]:
channels_dict = {'01':'banner',
                 '02':'text',
                 '06':'keyword',
                 '07':'link',
                 '08':'video',
                 '09':'mobile',
                 '10':'mobile',
                 '11':'mobile'}

In [9]:
columns = channels.copy()

In [11]:
columns.insert(0,"start")
columns

['start', 'banner', 'text', 'keyword', 'link', 'video', 'mobile', 'unknown']

In [12]:
columns.append("null")

In [13]:
columns

['start',
 'banner',
 'text',
 'keyword',
 'link',
 'video',
 'mobile',
 'unknown',
 'null']

In [14]:
columns.append("conversion")

In [15]:
columns

['start',
 'banner',
 'text',
 'keyword',
 'link',
 'video',
 'mobile',
 'unknown',
 'null',
 'conversion']